In [1]:
import numpy as np
import pybullet as p
import open3d as o3d
import assignment_6_helper as helper

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:


def get_antipodal(pcd):
    """
    function to compute antipodal grasp given point cloud pcd
    :param pcd: point cloud in open3d format (converted to numpy below)
    :return: gripper pose (4, ) numpy array of gripper pose (x, y, z, theta)
    """
    # convert pcd to numpy arrays of points and normals
    pc_points = np.asarray(pcd.points)
    pc_normals = np.asarray(pcd.normals)

    print(pc_points[0])

    # ------------------------------------------------
    # FILL WITH YOUR CODE

    # gripper orientation - replace 0. with your calculations
    theta = 0.
    # gripper pose: (x, y, z, theta) - replace 0. with your calculations
    gripper_pose = np.array([0.01654522, 0.0515762 , 0.04052635, theta])
    # ------------------------------------------------

    return gripper_pose

In [14]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

def find_object_centers(pcd, distance_threshold=0.05, min_points=50):
    """
    Find the centers of separate objects in the point cloud using DBSCAN clustering
    and return both the centroids and the segmented point clouds.

    :param pcd: open3d.geometry.PointCloud
    :param distance_threshold: float, the epsilon parameter for DBSCAN
    :param min_points: int, minimum number of points to form a cluster
    :return: 
        centroids: list of centroids as numpy arrays
        segmented_pcds: list of open3d.geometry.PointCloud objects for each cluster
    """
    # Optional Preprocessing: Downsample the point cloud for faster processing
    down_pcd = pcd.voxel_down_sample(voxel_size=0.005)  # Adjust voxel_size as needed

    # Optional Preprocessing: Remove statistical outliers
    cl, ind = down_pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    clean_pcd = down_pcd.select_by_index(ind)

    # Perform DBSCAN clustering
    labels = np.array(clean_pcd.cluster_dbscan(eps=distance_threshold, min_points=min_points, print_progress=True))

    if labels.size == 0:
        print("No clusters found.")
        return [], []

    max_label = labels.max()
    print(f"Point cloud has {max_label + 1} clusters")

    if max_label + 1 < 2:
        print("Less than two clusters found. Adjust DBSCAN parameters.")
        return [], []

    # Assume the two largest clusters are the objects
    unique_labels, counts = np.unique(labels, return_counts=True)
    sorted_indices = np.argsort(counts)[::-1]  # Descending order
    top_labels = unique_labels[sorted_indices[:2]]  # Top two clusters

    centroids = []
    segmented_pcds = []
    for label in top_labels:
        cluster_indices = np.where(labels == label)[0]
        cluster_points = np.asarray(clean_pcd.points)[cluster_indices]
        centroid = cluster_points.mean(axis=0)
        centroids.append(centroid)
        print(f"Centroid of cluster {label}: {centroid}")

        # Create a new PointCloud for the cluster
        cluster_pcd = clean_pcd.select_by_index(cluster_indices.tolist())
        segmented_pcds.append(cluster_pcd)

    return centroids, segmented_pcds


In [8]:
world = helper.World()

In [9]:
pcd = world.get_point_cloud()

In [10]:
pc_points = np.asarray(pcd.points)
pc_normals = np.asarray(pcd.normals)

In [11]:
pc_points

array([[0.0398135 , 0.17382749, 0.05139165],
       [0.04180412, 0.17382642, 0.05139245],
       [0.03774101, 0.17053292, 0.05138995],
       ...,
       [0.03874573, 0.1705783 , 0.00665988],
       [0.04168393, 0.1721433 , 0.00639289],
       [0.04015917, 0.17252699, 0.00701489]])

In [15]:
centroids, segmented_pcds = find_object_centers(pcd)
centroids

Point cloud has 2 clusters
Centroid of cluster 0: [-0.03388384 -0.02470299  0.03337826]
Centroid of cluster 1: [0.0660326  0.12496021 0.03402612]


[array([-0.03388384, -0.02470299,  0.03337826]),
 array([0.0660326 , 0.12496021, 0.03402612])]

In [18]:
delta_z=0.01

centroid = centroids[0]
object_points = np.asarray(segmented_pcds[0].points)
# Define the plane z = centroid_z with a tolerance
centroid_z = centroid[2]
mask = np.abs(object_points[:, 2] - centroid_z) < delta_z
points_on_plane = object_points[mask]
# if points_on_plane.shape[0] > 0:
#     # Compute distances in x and y from the centroid
#     distances_xy = np.linalg.norm(points_on_plane[:, :2] - centroid[:2], axis=1)
#     closest_idx = np.argmin(distances_xy)
#     grasp_point = points_on_plane[closest_idx]
#     print(f"Found point on plane for centroid {centroid}: {grasp_point}")
# else:
# If no points are exactly on the plane, find the point closest to the plane
z_diff = np.abs(object_points[:, 2] - centroid_z)
closest_z_idx = np.argmin(z_diff)
grasp_point = object_points[closest_z_idx]
print(f"No points within delta_z. Selected closest point to plane for centroid {centroid}: {grasp_point}")

No points within delta_z. Selected closest point to plane for centroid [-0.03388384 -0.02470299  0.03337826]: [-0.04999192 -0.00671114  0.03313947]


In [13]:
# terminate simulation environment once you're done!
p.disconnect()

In [5]:
n_tries = 1
# Initialize the world
world = helper.World()

# start grasping loop
# number of tries for grasping
for i in range(n_tries):
    # get point cloud from cameras in the world
    pcd = world.get_point_cloud()
    # check point cloud to see if there are still objects to remove
    finish_flag = helper.check_pc(pcd)
    if finish_flag:  # if no more objects -- done!
        print('===============')
        print('Scene cleared')
        print('===============')
        break
    # visualize the point cloud from the scene
    helper.draw_pc(pcd)
    # compute antipodal grasp
    gripper_pose = get_antipodal(pcd)
    # send command to robot to execute
    robot_command = world.grasp(gripper_pose)
    # robot drops object to the side
    world.drop_in_bin(robot_command)
    # robot goes to initial configuration and prepares for next grasp
    world.home_arm()
    # go back to the top!

# terminate simulation environment once you're done!
p.disconnect()

error: Only one local in-process GUI/GUI_SERVER connection allowed. Use DIRECT connection mode or start a separate GUI physics server (ExampleBrowser, App_SharedMemoryPhysics_GUI, App_SharedMemoryPhysics_VR) and connect over SHARED_MEMORY, UDP or TCP instead.